Importando Datasets

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, confusion_matrix, accuracy_score

Importando Dataset

In [39]:
dataset = pd.read_csv('BlackFriday.csv')

Tratamento dos Dados

In [40]:
#CATEGORIA DE CADA PRODUTO
dataset_categorical = dataset[['Product_ID', 'Product_Category_1', 'Product_Category_2', 'Purchase']].copy()

#PREENCHENDO NULOS DA CATEGORIA 2
produto2_random = pd.DataFrame(dataset_categorical['Product_Category_2'].value_counts(1))
produto2_random = produto2_random.reset_index()
dataset_categorical['Product_Category_2'].fillna(np.random.choice(produto2_random.index, p=produto2_random.Product_Category_2), inplace=True)

#DIVIDINDO VARIÁVEIS POR CATEGORIA
categorical1 = dataset_categorical['Product_Category_1']
categorical2 = dataset_categorical['Product_Category_2']


#STANDARD SCALER
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
dataset_easy = dataset[['User_ID', 'Gender', 'Age', 'Occupation', 'City_Category','Stay_In_Current_City_Years', 'Marital_Status', 'Purchase']].copy()

#LABEL ENCODER
labelencoder = LabelEncoder()
dataset_easy['Gender'] = labelencoder.fit_transform(dataset_easy['Gender'])

labelencoder = LabelEncoder()
dataset_easy['Age'] = labelencoder.fit_transform(dataset_easy['Age'])

labelencoder = LabelEncoder()
dataset_easy['Occupation'] = labelencoder.fit_transform(dataset_easy['Occupation'])

labelencoder = LabelEncoder()
dataset_easy['City_Category'] = labelencoder.fit_transform(dataset_easy['City_Category'])

labelencoder = LabelEncoder()
dataset_easy['Stay_In_Current_City_Years'] = labelencoder.fit_transform(dataset_easy['Stay_In_Current_City_Years'])

labelencoder = LabelEncoder()
dataset_easy['Marital_Status'] = labelencoder.fit_transform(dataset_easy['Marital_Status'])

dataset_easy2 = dataset_easy[['Gender', 'Age', 'Occupation', 'City_Category', 'Stay_In_Current_City_Years', 'Marital_Status']].copy()

"""
#QUAIS AS MELHORES VARIÁVEIS PARA PREDIZER A CATEGORIA 1?
from info_gain import info_gain
Gender  = info_gain.info_gain(dataset_easy2['Gender'], categorical1)
Age  = info_gain.info_gain(dataset_easy2['Age'], categorical1)
Occupation  = info_gain.info_gain(dataset_easy2['Occupation'], categorical1)
City_Category  = info_gain.info_gain(dataset_easy2['City_Category'], categorical1)
Stay_In_Current_City_Years  = info_gain.info_gain(dataset_easy2['Stay_In_Current_City_Years'], categorical1)
Marital_Status  = info_gain.info_gain(dataset_easy2['Marital_Status'], categorical1)
print('Ganho de Informação\n', Gender, Age, Occupation, City_Category, Stay_In_Current_City_Years, Marital_Status)

#Gender, Age e Occupation são os melhores preditores para prever categorical1
dataset_easy2 = dataset_easy2[['Gender', 'Age', 'Occupation']].copy()
"""



dataset_easy = dataset_easy.groupby('User_ID')['Gender', 'Age', 'Occupation', 'City_Category',
       'Stay_In_Current_City_Years', 'Marital_Status', 'Purchase'].mean()

X = dataset_easy[['Gender', 'Age', 'Occupation', 'City_Category', 'Marital_Status']].copy()

y = np.array(dataset_easy['Purchase'].copy())


"""
#ONEHOTENCODER
onehot = OneHotEncoder(categorical_features=[2, 4])
X = onehot.fit_transform(X).toarray()

onehot = OneHotEncoder(categorical_features=[2])
dataset_easy2 = onehot.fit_transform(dataset_easy2).toarray()
"""

'\n#ONEHOTENCODER\nonehot = OneHotEncoder(categorical_features=[2, 4])\nX = onehot.fit_transform(X).toarray()\n\nonehot = OneHotEncoder(categorical_features=[2])\ndataset_easy2 = onehot.fit_transform(dataset_easy2).toarray()\n'

In [21]:
X['Occupation'].value_counts()

4.0     740
0.0     688
7.0     669
1.0     517
17.0    491
12.0    376
14.0    294
20.0    273
2.0     256
16.0    235
6.0     228
10.0    192
3.0     170
15.0    140
13.0    140
11.0    128
5.0     111
9.0      88
19.0     71
18.0     67
8.0      17
Name: Occupation, dtype: int64

In [41]:

#ONEHOTENCODER
onehot = OneHotEncoder(categorical_features=[3])
X = onehot.fit_transform(X).toarray()

onehot = OneHotEncoder(categorical_features=[3])
dataset_easy2 = onehot.fit_transform(dataset_easy2).toarray()


c:\users\bruno\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
c:\users\bruno\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
c:\users\bruno\appdata\local\programs\python\python36\lib\

Recomendando Categoria 2

In [4]:
"""
Relação em porcentagem entre Categoria 1 e Categoria 2
"""

vetor = []

for i in range(1, len((dataset_categorical['Product_Category_1'].unique()))):

    k = dataset_categorical['Product_Category_2'][dataset_categorical['Product_Category_1']==i]
    k=k.value_counts(1)

    vetor.append([i, k.index, k.values])


insere = int(input())

for i in range(len(vetor)):
    if(vetor[i][0]==insere):
        df = pd.DataFrame({'Produto':vetor[i][1], 'Probabilidade':(vetor[i][2]*100).round(2)})
        print(df)

2
    Produto  Probabilidade
0       4.0          35.40
1       5.0          18.15
2       3.0          12.03
3       8.0          11.41
4       9.0           6.59
5      15.0           5.88
6      17.0           3.84
7      16.0           2.35
8      14.0           2.13
9       6.0           0.82
10     18.0           0.78
11     12.0           0.63


"\nfor i in range(len(dataset_categorical)):\n    vetor.append(dataset_categorical['Product_Category_1'][dataset_categorical['Product_Category_1']==1])\n"

In [31]:
#Teste de Significância
OLS = sm.OLS(exog=X, endog=y).fit()
print(OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.886
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     9148.
Date:                Fri, 06 Sep 2019   Prob (F-statistic):               0.00
Time:                        15:36:42   Log-Likelihood:                -56186.
No. Observations:                5891   AIC:                         1.124e+05
Df Residuals:                    5886   BIC:                         1.124e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Gender          3661.2867     87.489     41.

In [45]:
X = pd.DataFrame(X)
y = pd.Series(y)
y.index

RangeIndex(start=0, stop=5891, step=1)

In [46]:




X = X.reset_index(drop=True)
#REMOVENDO OUTLIERS
max = y.quantile(0.75)+(y.quantile(0.75)-y.quantile(0.25))*1.5
min = y.quantile(0.25)+(y.quantile(0.75)-y.quantile(0.25))*1.5

print(max)

for i in range(len(y)):
       if((y[i]>max) or (y[i]<min)):
            y = y.drop(i, axis=0)
            X = X.drop(i, axis=0)

14495.768602421325


In [55]:
#DIVIDINDO O DATASET
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#ADABOOST
from sklearn.ensemble import AdaBoostRegressor
ada = AdaBoostRegressor(learning_rate=0.1, loss='linear', n_estimators=600)
ada.fit(X_train, y_train)

y_pred_ada = ada.predict(X_test)

#MÉTRICAS

print('RMSE', np.sqrt(mean_squared_error(y_test, y_pred_ada)))

RMSE/mean(y_test) 710.1734813391115
